# NLPaugを用いたデータ拡張

このノートブックでは、文字レベルと単語レベルでのデータ拡張を試します。他のタイプのデータ拡張として文や音声、スペクトラムなどもあります。詳細については、[GitHubリポジトリ](https://github.com/makcedward/nlpaug)や[公式ドキュメント](https://nlpaug.readthedocs.io/en/latest/)を参照してください。

## 準備
### パッケージのインストール

In [27]:
!pip install nlpaug==1.1.4

### パッケージのインポート

In [3]:
from pprint import pprint

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

### テキストの用意

このノートブックでは、以下のテキストを対象に、データ拡張を試します。

In [2]:
text = "The quick brown fox jumps over the lazy dog ."

## データ拡張
### 文字レベルでの拡張

文字レベルのデータ拡張手法として、以下の2つを試します。

1. OCR Augmenter: OCRを用いて画像からテキストを抽出すると、`o`ではなく`0`、`z`ではなく`2`を抽出するといった誤りが発生します。このようなOCRで発生する誤りを再現することで、データ拡張を行います。
2. Keyboard Augmenter: タイピング中に文字を打ち間違えたことのない人はいないでしょう。このデータ拡張手法では、単語内の文字をキーボード上の同じ距離にある文字に置き換えることで、タイピングの誤りをシミュレートします。



In [16]:
aug = nac.OcrAug()
# n=3を指定すると、拡張したテキストを3つ生成します。
augmented_texts = aug.augment(text, n=3)

print("Original:")
print(text)
print("Augmented Texts:")
pprint(augmented_texts)

Original:
The quick brown fox jumps over the lazy dog .
Augmented Texts:
['The quick brown fux jomps over the lazy dog.',
 'The qoick 6rown fox jumps over the lazy d0g.',
 'The quicr brown fox jumps uver the lazy dog.']


In [15]:
aug = nac.KeyboardAug()
augmented_text = aug.augment(text, n=3)

print("Original:")
print(text)
print("Augmented Text:")
pprint(augmented_text)

Original:
The quick brown fox jumps over the lazy dog .
Augmented Text:
['The qu*ck brPwn fox jumps ovfr the lazy dog.',
 'The quicu brown fox jumpw over the lazh dog.',
 'The quick brown fox jjmps oger the laz5 dog.']


### 単語レベルでの拡張

単語レベルでの拡張方法として、辞書やword2vecを使って、類似する単語の挿入や置き換えをしてみましょう。

#### Spelling augmentor

スペルミスの含まれる単語を置き換えることで、拡張してみましょう。

In [24]:
# 辞書のダウンロード
!wget https://raw.githubusercontent.com/makcedward/nlpaug/master/nlpaug/res/word/spelling/spelling_en.txt

--2021-07-02 03:11:56--  https://raw.githubusercontent.com/makcedward/nlpaug/master/nlpaug/res/word/spelling/spelling_en.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 543624 (531K) [text/plain]
Saving to: ‘spelling_en.txt’

spelling_en.txt     100%[===================>] 530.88K  --.-KB/s    in 0.04s   

2021-07-02 03:11:56 (12.3 MB/s) - ‘spelling_en.txt’ saved [543624/543624]



In [25]:
!head spelling_en.txt

hierarchically hierachically
esteem estime esstem
luck lock lucke luke like Lock lake look luch
cleaning cleanig cliner
deteriorated dereriorates
girls gherls girles gerls girs grils gairls
millionaire millionare millionnaire milionaire millioner milionary millionier millonaire billinaire
ancient acient ancien anciest accient
Plina P?lina
engrossed engresed


In [26]:
aug = naw.SpellingAug('spelling_en.txt')
augmented_texts = aug.augment(text)

print("Original:")
print(text)
print("Augmented Texts:")
print(augmented_texts)

Original:
The quick brown fox jumps over the lazy dog .
Augmented Texts:
quick brown fox jumps over lazy dog.


#### Word embeddings augmentor

In [8]:
#Downloading the reqired model
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-07-02 02:53:21--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.113.93
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.113.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  50.7MB/s    in 32s     

2021-07-02 02:53:53 (49.3 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [9]:
!gunzip GoogleNews-vectors-negative300.bin.gz

In [10]:
!ls

GoogleNews-vectors-negative300.bin  sample_data  spelling_en.txt


単語埋め込みの類似度に基づいて、単語をランダムに挿入してみましょう。

In [11]:
# model_type: word2vec, glove or fasttext
aug = naw.WordEmbsAug(
    model_type='word2vec',
    model_path='GoogleNews-vectors-negative300.bin',
    action="insert"
)
augmented_text = aug.augment(text)

print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
The quick brown fox jumps over the lazy dog .
Augmented Text:
The GCPP quick Trippi brown fox jumps over the lazy Gillig dog.


word2vecの類似度に基づいて、単語を置き換えてみましょう。


In [12]:
aug = naw.WordEmbsAug(
    model_type='word2vec',
    model_path='GoogleNews-vectors-negative300.bin',
    action="substitute"
)
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
The quick brown fox jumps over the lazy dog .
Augmented Text:
The quick brown fox jump whopping clearly lazy dog.
